# Airnow Monitoring Site ESDR Device and Feed Cache

Downloads the ESDR devices and feeds representing the known Airnow monitoring site locations and caches as a JSON file.  Having a local cache is useful for speeding up uploads.  Used by cocalc scripts, and in user-facing visualizations.

Reports to stat.createlab.org as `Airnow Monitoring Site ESDR Device and Feed Cache`.

In [0]:
import json, os, dateutil, re, requests, subprocess, datetime, glob, stat

from dateutil import rrule, tz, parser

In [0]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb


def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


exec_ipynb('./python-utils/utils.ipynb')
exec_ipynb('./python-utils/esdr-library.ipynb')
exec_ipynb('./airnow-common.ipynb')

In [0]:
MIRROR_TIME_PERIOD_SECS = 60 * 5   # every 5 minutes

STAT_SERVICE_NAME = 'Airnow Monitoring Site ESDR Device and Feed Cache'
STAT_HOSTNAME = 'hal21'
STAT_SHORTNAME = 'airnow-monitoring-site-esdr-device-and-cache'

JSON_FILENAME = 'esdr_monitoring_site_location_devices_and_feeds.json'

In [0]:
Stat.set_service(STAT_SERVICE_NAME)

In [0]:
esdr = None
airnow_product = None

In [0]:
def get_airnow_product():
    try:
        global esdr, airnow_product
        if not esdr:
            esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
        if not airnow_product:
            # esdr.create_product('AirNow', 'AirNow', 'EPA and Sonoma Tech', 'Real-time feeds from EPA/STI AirNow')
            airnow_product = esdr.get_product_by_name('AirNow')
        return airnow_product
    except requests.HTTPError as e:
        Stat.warning('Failed to get Airnow ESDR product due to error: %s' % (str(e)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        return None

#get_airnow_product()

In [0]:
def refresh_esdr_monitoring_site_devices_cache():
    global esdr, airnow_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not airnow_product:
        airnow_product = get_airnow_product()

    if not airnow_product:
        Stat.warning('No ESDR Airnow product found!', host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        return False

    # get all ESDR devices belonging to the Airnow product, dealing with multiple pages of data if necessary
    devices = []
    while True:
        try:
            response = esdr.api('GET', '/api/v1/devices', {'where':'productId='+str(airnow_product['id']), 'fields':'id,name,serialNumber', 'offset':len(devices)})
            if 'data' in response and 'rows' in response['data']:
                new_rows = response['data']['rows']
                devices.extend(new_rows)
                if len(devices) == response['data']['totalCount'] or len(new_rows) <= 0:
                    break
            else:
                raise Exception("No data in response when fetching ESDR devices")
        except requests.HTTPError as e:
            Stat.warning('Failed to fetch devices for %s due to error: %s' % (JSON_FILENAME, str(e)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            return False
        except Exception as e:
            Stat.warning('Failed to fetch devices for %s due to error: %s' % (JSON_FILENAME, str(e)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            return False

    # get all ESDR feeds belonging to the Airnow product, dealing with multiple pages of data if necessary
    feeds = []
    while True:
        try:
            response = esdr.api('GET', '/api/v1/feeds', {'where':'productId='+str(airnow_product['id']), 'fields':'id,deviceId,latitude,longitude', 'orderBy':'deviceId,-id', 'offset':len(feeds)})
            if 'data' in response and 'rows' in response['data']:
                new_rows = response['data']['rows']
                feeds.extend(new_rows)
                if len(feeds) == response['data']['totalCount'] or len(new_rows) <= 0:
                    break
            else:
                raise Exception("No data in response when fetching ESDR feeds")
        except requests.HTTPError as e:
            Stat.warning('Failed to fetch feeds for %s due to error: %s' % (JSON_FILENAME, str(e)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            return False
        except Exception as e:
            Stat.warning('Failed to fetch feeds for %s due to error: %s' % (JSON_FILENAME, str(e)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            return False

    # create a map, which maps serial number to device id and name
    esdr_device_map = {}
    device_id_to_serial_number_map = {}
    for device in devices:
        esdr_device_map[device['serialNumber']] = {'id':device['id'], 'name':device['name'], 'feeds':[]}
        device_id_to_serial_number_map[device['id']] = device['serialNumber']

    # iterate over all the feeds and insert into the esdr_device_map
    for feed in feeds:
        device_id = feed['deviceId']
        if device_id in device_id_to_serial_number_map:
            serial_number = device_id_to_serial_number_map[device_id]
            device = esdr_device_map[serial_number]
            device['feeds'].append({'id': feed['id'], 'lat': feed['latitude'], 'lng': feed['longitude']})
        else:
            Stat.warning('Device ID [%s] not found in device_id_to_serial_number_map!' % (device_id), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            return False

    # now write the JSON file
    json_dest = AirnowCommon.DATA_DIRECTORY + '/' + JSON_FILENAME
    tmp = json_dest + '.tmp' + str(os.getpid())
    os.makedirs(os.path.dirname(tmp), exist_ok=True)
    with open(tmp, 'w') as json_file:
        json.dump(esdr_device_map, json_file, sort_keys=True)
    os.rename(tmp, json_dest)

    # make the JSON file readable by everyone
    os.chmod(json_dest, stat.S_IREAD | stat.S_IWRITE | stat.S_IRGRP | stat.S_IROTH)

    Stat.info('Successfully cached %d devices and %d feeds to %s' % (len(devices), len(feeds), JSON_FILENAME), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    return True

In [0]:
def run():
    starting_timestamp = datetime.datetime.now().timestamp()
    Stat.info('Downloading Airnow monitoring site ESDR devices and feeds and caching to %s...' % JSON_FILENAME, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    was_successful = refresh_esdr_monitoring_site_devices_cache()
    elapsed_seconds = datetime.datetime.now().timestamp() - starting_timestamp
    if was_successful:
        Stat.up('Done! (elapsed time: %d seconds)' % (elapsed_seconds), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    else:
        Stat.down('Failed to download Airnow monitoring site ESDR devices and cache to %s (elapsed time: %d seconds)' % (JSON_FILENAME, elapsed_seconds), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

def run_forever():
    while True:
        run()
        sleep_until_next_period(MIRROR_TIME_PERIOD_SECS)

# run_forever()
run()